# PGF5005 — Mecânica Clássica · Itens **4.8, 4.9 e 4.10**  
**Comparações: Euler simplético × Euler convencional**

- **4.8** — Refaça os gráficos \(q(t)\times t\) e \(p(t)\times t\) para **librar** e **rotacionar** com \(\Delta t=\{10^{-1},10^{-2},10^{-3}\}\), agora comparando **Euler simplético** com **Euler convencional**.  
- **4.9** — Refaça \(E(t)\times t\) para os mesmos casos e compare.  
- **4.10** — Em analogia ao 4.5, construa os **diagramas de fase \(p(t)\times q(t)\)** e compare.

> Convenção nos gráficos: **linha cheia** = *Euler simplético* (Euler–Cromer); **tracejada** = *Euler convencional*.


## 1) Imports e modelo do pêndulo

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt

# === Modelo do pêndulo adimensional ===
def dU_dq(q):   # U(q) = -cos q
    return math.sin(q)

def H(q, p):
    return 0.5*p*p - math.cos(q)


## 2) Integradores: Euler **convencional** e Euler **simplético**

In [ ]:
# --- Euler convencional ---
def euler_conv_step(q, p, dt):
    q_next = q + dt * p          # usa p^n
    p_next = p - dt * dU_dq(q)   # usa q^n
    return q_next, p_next

def integrar_euler_conv(q0, p0, dt, T):
    n = int(np.ceil(T/dt))
    t = np.empty(n+1); q = np.empty(n+1); p = np.empty(n+1); E = np.empty(n+1)
    t[0]=0.0; q[0]=q0; p[0]=p0; E[0]=H(q0,p0)
    for k in range(n):
        q[k+1], p[k+1] = euler_conv_step(q[k], p[k], dt)
        t[k+1]=(k+1)*dt
        E[k+1]=H(q[k+1], p[k+1])
    return t,q,p,E

# --- Euler simplético (Euler–Cromer) ---
def euler_symp_step(q, p, dt):
    q_next = q + dt * p              # usa p^n
    p_next = p - dt * dU_dq(q_next)  # usa q^{n+1}
    return q_next, p_next

def integrar_euler_symp(q0, p0, dt, T):
    n = int(np.ceil(T/dt))
    t = np.empty(n+1); q = np.empty(n+1); p = np.empty(n+1); E = np.empty(n+1)
    t[0]=0.0; q[0]=q0; p[0]=p0; E[0]=H(q0,p0)
    for k in range(n):
        q[k+1], p[k+1] = euler_symp_step(q[k], p[k], dt)
        t[k+1]=(k+1)*dt
        E[k+1]=H(q[k+1], p[k+1])
    return t,q,p,E


## 3) Utilitários de simulação e gráficos

In [ ]:
def simular_tres_dts(integrador, q0, p0, T, dt_list):
    resultados = {}
    for dt in dt_list:
        resultados[dt] = integrador(q0, p0, dt, T)
    return resultados

def plot_duplo(series_A, series_B, xlab, ylab, title, save=None):
    """Plota A (cheio) e B (tracejado) com 3 curvas cada.
    series_*: lista de (x, y, label).
    save: caminho para salvar PNG (opcional).
    """
    plt.figure()
    for x,y,label in series_A:
        plt.plot(x,y,label=label, lw=1.2)
    for x,y,label in series_B:
        plt.plot(x,y,label=label, lw=1.2, linestyle='--')
    plt.xlabel(xlab); plt.ylabel(ylab); plt.title(title + '\n(cheio: simplético · tracejado: conv.)')
    plt.legend()
    if save:
        plt.savefig(save, dpi=200, bbox_inches='tight')
    plt.show()

def plot_fase_duplo(series_A, series_B, title, save=None):
    """Series_*: lista de (q, p, label)."""
    plt.figure()
    for q,p,label in series_A:
        plt.plot(q,p,label=label, lw=1.2)
    for q,p,label in series_B:
        plt.plot(q,p,label=label, lw=1.2, linestyle='--')
    plt.xlabel('q(t)'); plt.ylabel('p(t)'); plt.title(title + '\n(cheio: simplético · tracejado: conv.)')
    plt.legend()
    if save:
        plt.savefig(save, dpi=200, bbox_inches='tight')
    plt.show()


## 4) Configuração dos cenários

In [ ]:
dt_list = [1e-1, 1e-2, 1e-3]
T = 30.0

# condições iniciais
ics = {
    'Librar (q0=0.5, p0=0.0)': (0.5, 0.0),
    'Rotacionar (q0=0.0, p0=2.2)': (0.0, 2.2),
}


## 5) **Item 4.8** — q(t) × t e p(t) × t (comparação)

In [ ]:
for nome,(q0,p0) in ics.items():
    res_symp = simular_tres_dts(integrar_euler_symp, q0, p0, T, dt_list)
    res_conv = simular_tres_dts(integrar_euler_conv, q0, p0, T, dt_list)

    series_q_symp = [(t,q,f"symp dt={dt:g}") for dt,(t,q,p,E) in res_symp.items()]
    series_q_conv = [(t,q,f"conv dt={dt:g}") for dt,(t,q,p,E) in res_conv.items()]
    plot_duplo(series_q_symp, series_q_conv, 't', 'q(t)', f'4.8 — {nome}: q(t) com 3 passos', save=None)

    series_p_symp = [(t,p,f"symp dt={dt:g}") for dt,(t,q,p,E) in res_symp.items()]
    series_p_conv = [(t,p,f"conv dt={dt:g}") for dt,(t,q,p,E) in res_conv.items()]
    plot_duplo(series_p_symp, series_p_conv, 't', 'p(t)', f'4.8 — {nome}: p(t) com 3 passos', save=None)


## 6) **Item 4.9** — Energia E(t) × t (comparação)

In [ ]:
for nome,(q0,p0) in ics.items():
    res_symp = simular_tres_dts(integrar_euler_symp, q0, p0, T, dt_list)
    res_conv = simular_tres_dts(integrar_euler_conv, q0, p0, T, dt_list)

    series_E_symp = [(t,E,f"symp dt={dt:g}") for dt,(t,q,p,E) in res_symp.items()]
    series_E_conv = [(t,E,f"conv dt={dt:g}") for dt,(t,q,p,E) in res_conv.items()]
    plot_duplo(series_E_symp, series_E_conv, 't', 'E(t)', f'4.9 — {nome}: Energia com 3 passos', save=None)


## 7) **Item 4.10** — Diagramas de fase p(t) × q(t) (comparação)

In [ ]:
for nome,(q0,p0) in ics.items():
    res_symp = simular_tres_dts(integrar_euler_symp, q0, p0, T, dt_list)
    res_conv = simular_tres_dts(integrar_euler_conv, q0, p0, T, dt_list)

    series_phase_symp = [(q,p,f"symp dt={dt:g}") for dt,(t,q,p,E) in res_symp.items()]
    series_phase_conv = [(q,p,f"conv dt={dt:g}") for dt,(t,q,p,E) in res_conv.items()]
    plot_fase_duplo(series_phase_symp, series_phase_conv, f'4.10 — {nome}: Fase p×q com 3 passos', save=None)


---
## 8) O que comentar no relatório

- O **Euler simplético** mantém **quase-conservação de energia**: \(E(t)\) oscila em torno de um valor médio
  (sem *drift* sistemático), sobretudo para \(\Delta t\) pequenos.
- O **Euler convencional** tende a apresentar **deriva** da energia ao longo do tempo (crescimento/decrescimento
  monotônico), que aparece nos gráficos de \(E(t)\), e se traduz em distorções nas curvas de \(q(t)\), \(p(t)\) e também no
  diagrama de fase \(p\times q\) (espirais que se afastam/aproximam do nível de energia).
- Ao reduzir \(\Delta t\) de \(10^{-1}\) para \(10^{-3}\), as discrepâncias se reduzem e as duas integrações convergem.
